In [ ]:
import pandas as pd

LOADING DATASET 

In [ ]:
movies = pd.read_csv(r"C:\\Users\\Himika\\Desktop\\python\\data_science_project\\movies.csv")
ratings = pd.read_csv(r"C:\\Users\\Himika\\Desktop\\python\\data_science_project\\ratings.csv")

In [ ]:
print(movies.head())
print(ratings.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510


In [35]:
movies[movies['title'].str.contains("waiting to", case=False, na=False)]

,movieId,title,genres
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance


In [ ]:
movie_data = pd.merge(ratings, movies, on='movieId')

In [ ]:
print(movie_data.head())

   userId  movieId  rating   timestamp  \
0       1      296     5.0  1147880044   
1       1      306     3.5  1147868817   
2       1      307     5.0  1147868828   
3       1      665     5.0  1147878820   
4       1      899     3.5  1147868510   

                                              title  \
0                               Pulp Fiction (1994)   
1  Three Colors: Red (Trois couleurs: Rouge) (1994)   
2  Three Colors: Blue (Trois couleurs: Bleu) (1993)   
3                                Underground (1995)   
4                        Singin' in the Rain (1952)   

                        genres  
0  Comedy|Crime|Drama|Thriller  
1                        Drama  
2                        Drama  
3             Comedy|Drama|War  
4       Comedy|Musical|Romance  


In [ ]:
# Replace '|' with space for better vectorization
movie_data['genres'] = movie_data['genres'].str.replace('|', ' ', regex=False)

# Drop duplicates to keep one row per movie
movies_unique = movie_data.drop_duplicates(subset='movieId')[['movieId', 'title', 'genres']]

print(movies_unique.head())

   movieId                                             title  \
0      296                               Pulp Fiction (1994)   
1      306  Three Colors: Red (Trois couleurs: Rouge) (1994)   
2      307  Three Colors: Blue (Trois couleurs: Bleu) (1993)   
3      665                                Underground (1995)   
4      899                        Singin' in the Rain (1952)   

                        genres  
0  Comedy Crime Drama Thriller  
1                        Drama  
2                        Drama  
3             Comedy Drama War  
4       Comedy Musical Romance  


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Fit and transform the genres
tfidf_matrix = tfidf.fit_transform(movies_unique['genres'])

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Check the shape (should be [num_movies x num_movies])
print(cosine_sim.shape)

(59047, 59047)


In [ ]:
# Reset index to keep track of movie positions
movies_unique = movies_unique.reset_index(drop=True)

# Create a mapping from movie titles to index
title_to_index = pd.Series(movies_unique.index, index=movies_unique['title'])

# Recommendation function
def recommend_movie(title, cosine_sim=cosine_sim):
    if title not in title_to_index:
        return ["Movie not found. Try a different title."]
    
    idx = title_to_index[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:11]  # top 10 excluding itself
    movie_indices = [i[0] for i in sim_scores]
    return movies_unique['title'].iloc[movie_indices].tolist()

# Example usage
print(recommend_movie('Toy Story (1995)'))

['Antz (1998)', 'Toy Story 2 (1999)', "Emperor's New Groove, The (2000)", 'Monsters, Inc. (2001)', 'Shrek the Third (2007)', 'Boxtrolls, The (2014)', 'Moana (2016)', 'Turbo (2013)', 'Adventures of Rocky and Bullwinkle, The (2000)', 'DuckTales: The Movie - Treasure of the Lost Lamp (1990)']


In [ ]:
def recommend_by_genre1(genre_name):
    # Find movies containing the specified genre
    similar_movies = movies_unique[movies_unique['genres'].str.contains(genre_name, case=False, na=False)]

    if similar_movies.empty:
        return ["No movies found for this genre."]

    return similar_movies['title'].head(10).tolist()


In [ ]:
print(recommend_by_genre1('Thriller'))

['Pulp Fiction (1994)', 'M (1931)', 'Gattaca (1997)', 'City of God (Cidade de Deus) (2002)', 'Bad Education (La mala educación) (2004)', 'Clear and Present Danger (1994)', 'True Lies (1994)', 'Fugitive, The (1993)', 'Jurassic Park (1993)', 'Rock, The (1996)']
